In [1]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [2]:
import os
os.chdir("/project/work/Passenger_Demand")

## 패키지 설치
!pip install seaborn
!pip install haversine
!pip install pyarrow
!pip install multiprocessing_on_dill
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import math
import numbers

from haversine import haversine
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'Nanum Gothic'
sns.set(font="NanumGothic")

In [14]:
%%time
bus_demand_401 = pd.read_parquet('model_mr/mybicard_401_agg.parquet', engine='pyarrow')

CPU times: user 40.9 ms, sys: 54 ms, total: 94.9 ms
Wall time: 26 ms


In [15]:
bus_demand_401.sort_values('transdate').head(20)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
223368,3101482,2020-04-08 00:00:00,1,1,0,2
16084,3100144,2020-04-08 00:00:00,2,0,0,2
89825,3101414,2020-04-08 00:00:00,0,1,0,1
164705,3101446,2020-04-08 00:00:00,1,0,0,1
156351,3101442,2020-04-08 00:00:00,1,0,0,1
95714,3101416,2020-04-08 00:00:00,1,0,0,1
150970,3101440,2020-04-08 00:00:00,1,0,0,1
278482,3101537,2020-04-08 05:00:00,2,0,0,2
293924,3101542,2020-04-08 05:00:00,16,0,0,16
13909,3100101,2020-04-08 05:00:00,1,0,0,1


In [16]:
bus_demand_401_10m['transdate'].min()

Timestamp('2020-04-08 00:00:00')

In [17]:
bus_demand_401_10m['transdate'].max()

Timestamp('2020-11-30 23:00:00')

In [12]:
bus_demand_401_10m['transdate'][0].month

2020

In [ ]:
def get_entire_period(df):
    start_date, end_date = df['transdate'].min(), df['transdate'].max()
    

In [ ]:
def fill_blank_data():
    

In [ ]:
from datetime import datetime, timedelta # 20180501 부터 10일간 데이터 반환 
base = datetime.strptime('20180501', '%Y%m%d') numdays = 10 date_list = [(base + timedelta(days=x)).strftime('%Y%m%d') for x in range(0, numdays)] print(date_list)

출처: https://118k.tistory.com/686 [개발자로 살아남기]

import datetime

a=datetime.datetime.now()-datetime.timedelta(days=1)#하루전
a=datetime.datetime.now()-datetime.timedelta(hours=1)#한시간전
a=datetime.datetime.now()-datetime.timedelta(weeks=1)#일주일전

 

2.달,년

import dateutil.relativedelta import relativedelta
a=datetime.datetime.now()-relativedelta(months=1)#한달전
a=datetime.datetime.now()-relativedelta(years=1)#일년전

In [7]:
import datetime
from functools import partial
from tqdm import tqdm

In [8]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수

        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key

        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
    #func = partial(func, **params)
    
    def map_func(data):
        return data.apply(func, axis = 1, group_data= data.copy() ,**params)
        
        
    with Pool(num_cores) as p:

        pd_result = pd.concat(p.map(map_func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [35]:
def period_before_cnt(data, group_data):
    data_copy = data.copy()
    col_names = ['totalcnt','normalcnt','studentcnt','childcnt']
    
    subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()
    before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=1))][col_names].copy()
    before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=1))][col_names].copy()
    
    j = 0
    while len(before_1day) == 0:
        j += 1
        if subset_df['transdate'].min() >= data_copy['transdate'] - datetime.timedelta(days=j):
            before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'])][col_names].copy()
            j = -1
            break
        before_1day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=j))][col_names].copy()
    before_1day['check'] = j
    
    k = 0
    while len(before_1week) == 0:
        k += 1
        if subset_df['transdate'].min() >= data_copy['transdate'] - datetime.timedelta(weeks=k):
            before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'])][col_names].copy()
            k = -1
            break
        before_1week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=k))][col_names].copy()
    before_1week['check'] = k
    
    for name in col_names + ['check']:
        data_copy[name + 'D-1'] = before_1day.reset_index(drop=True).loc[0][name]
        data_copy[name + 'W-1'] = before_1week.reset_index(drop=True).loc[0][name]
    return data_copy

In [36]:
bus_demand_401_10m.shape

(401746, 125)

In [37]:
%%time
bb = parallelize_dataframe(df = bus_demand_401_10m[['stop_id', 'transdate', 'totalcnt', 'normalcnt', 'studentcnt', 'childcnt']]
                                          , func = period_before_cnt, group_keys = 'stop_id', num_cores = 10)

CPU times: user 3.34 s, sys: 3.81 s, total: 7.15 s
Wall time: 52min 3s
